# 读写文件
为了实时保存结果，我们需要学会如何加载和储存权重向量和整个模型。

## 加载和保存张量
对于单个张量。我们可以直接调用`load`和`save`函数分别读写他们。这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。

In [1]:
import torch 
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

现在我们可以将储存在文件中的数据读回内存。

In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

可以存储一个张量列表，然后将他们读回内存。

In [3]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

甚至可以 **写入和读取从字符串映射到张量的字典**。当我们要读取或写入模型中的所有权重时，这很方便。

In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## **加载和保存模型参数**
深度学习框架提供了内置函数来保存和加载整个网络。需要注意的一个细节是，这将保存参数而`不是整个模型`。例如，如果我们有一个3层多层感知机，我们需要单独指定架构。因为模型本身可以包含藤椅代码，所以模型本身难以序列化。因此为了恢复模型，我们需要用代码生成架构，然后从磁盘中加载参数。

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

接下来，将模型的参数存储在一个叫做`mlp.params`的文件中。

In [6]:
torch.save(net.state_dict(), 'mlp.params')


为了恢复模型，我们实例化了一个原始多层感知机模型中的一个备份。这里我们不需要随机初始化模型参数，而是直接读取文件中存储的参数

In [7]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval

<bound method Module.eval of MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)>

由于两个实例具有相同的模型参数，在输入相同的`X`时，两个实例的计算结果应该相同。

验证一下

In [8]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])